# A Machine Learning approach for Sentiment Analysis for Italian Reviews in Healthcare

This notebook was used to collect the dataset of 47224 Italian reviews in the Healthcare domain. Data were collected in date May, 26th 2020.

## IMPORT


In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

## CONSTANTS

In [2]:
path = '' ### Insert your path
main_url = "https://www.qsalute.it"
c_url = main_url + '/patologie/'

## SCRAPER CLASS

In [3]:
class Scraper():
  def __init__(self, url):
    self.url = url
    # Create a session
    rS = requests.Session()
    try:
      self.page = rS.get(url)
    except Exception as e:
      print(e, self.page)
    # Create a BeautifulSoup object
    self.soup = BeautifulSoup(self.page.text, 'html.parser')
    self.tags = []
    self.name = ""
    self.link = ""
    self.class_list = []
    self.item = ""
    self.name_list = []
    self.item_list = []

  def req_tags(self, classe, tag=None, attributes=None):
    self.class_list = self.soup.find(class_=classe)
    if tag != None:
      self.tags = self.class_list.find_all(tag, attributes)
      return self.tags
    return self.class_list

  def req_item(self, item, tags=None, ifname=""):
    if tags == None:
      tags = self.tags
    for tag in tags:
      name = tag.contents[0]
      if ifname != "":
        if name == ifname:
          self.item_list.append(tag.get(item))
          self.name_list.append(str(name).replace(u'<span>', u'').replace(u'</span>', u''))
      else:
        self.item_list.append(tag.get(item))
        self.name_list.append(str(name).replace(u'<span>', u'').replace(u'</span>', u''))
    return self.name_list, self.item_list

  def get_item(self):
    return self.name, self.item

  def reset(self):
    self.name_list = []
    self.item_list = []
    self.tags = []

  def req_single_item(self, item, tags=None, ifname=""):
    if tags == None:
      tags = self.tags
    for tag in tags:
      self.name = tag.contents[0]
      if ifname != "":
        if self.name == ifname:
          self.item = tag.get(item)
        else:
          self.item = ""
      else:
        self.item = tag.get(item)
    return self.name, self.item

## REVIEWS CLASS

In [4]:
class Review():
  def __init__(self, url, dname):
    self.categoria = dname
    self.reparto = ""  # REPARTO (Senologia Campus Bio-Medico Roma)          / Struttura (es. Policlinico Campus Bio-Medico Roma)
    self.struttura = ""   # STRUTTURA (es. Policlinico Campus Bio-Medico Roma)  / Categoria (es. ospedali e case di cura di Roma)
    self.data = ""
    self.autore= ""
    self.rating = {}
    self.titolo = ""
    self.testo = ""
    self.patologia = ""
    self.scraper = Scraper(url)

  def set_review(self):
    self.scraper.req_tags('jrContentTitle', 'a')
    self.reparto = self.scraper.req_single_item('href')[0]

    self.scraper.req_tags('jrListingInfo', tag='span')
    self.struttura = self.scraper.req_single_item('span')[0]

    self.scraper.req_tags('jrReviewInfo', tag='time')
    self.data = self.scraper.req_single_item('time')[0]

    self.scraper.req_tags('jrUserInfo', tag='span')
    self.autore = self.scraper.req_single_item('span')[0]

    keys = self.scraper.req_tags('jrRatingInfo', tag='div', attributes={"class": 'jrCol jrRatingLabel'})
    values = self.scraper.req_tags('jrRatingInfo', tag='div', attributes={"class": 'jrCol jrRatingValue'})
    keys = self.scraper.req_item('jrCol jrRatingLabel', tags=keys)[0]
    self.scraper.reset()
    values = self.scraper.req_item('jrCol jrRatingValue', tags=values)[0]
    for i, key in enumerate(keys):
      self.rating[key.replace(u'\xa0', u'')] = float(values[i])

    self.scraper.req_tags('jrReviewContent', tag='h4', attributes={"class": 'jrReviewTitle'})
    self.titolo= self.scraper.req_single_item('h4')[0]

    self.scraper.req_tags('jrReviewContent', tag='div', attributes={"class": 'description jrReviewComment'})
    text = self.scraper.req_single_item('p')[0]
    self.testo = str(text).replace('<p>', u'').replace(u'</p>', u'').replace(u'<br/>', u'')

    self.scraper.req_tags('jrCustomFields', tag='div', attributes={"class": 'jrFieldValue'})
    self.patologia = self.scraper.req_single_item('div')[0]

    return self.get_df_review()

  def get_df_review(self):
    df = pd.DataFrame([vars(self)])
    return df.drop(['scraper'], axis=1)

## SCRAPING DISEASE CATEGORIES PAGE

Choose a category:


1.   Malattie del cervello e del sistema nervoso
2.   Malattie della pelle
3.   Malattie del cuore
4.   Malattie di urologia
5.   Malattie di ginecologia e ostetricia
6.   Malattie di senologia e ginecologia oncologica
7.   Malattie di oncologia e radioterapia
8.   Malattie di ematologia
9.   Malattie di reumatologia
10.  Malattie di pneumologia e allergologia
11.  Malattie di gastroenterologia, epatologia e endoscopia
12.  Malattie di chirurgia vascolare
13.  Malattie di chirurgia toracica
14.  Malattie di chirurgia maxillo-facciale
15.  Malattie di chirurgia plastica
16.  Malattie di oculistica
17.  Malattie infettive
18.  Malattie di otorinolaringoiatria e audiologia
19.  Malattie di endocrinologia e diabetologia

In [5]:
cScraper = Scraper(c_url)
cScraper.req_tags('item-page', 'a')
name, links = cScraper.req_item('href')

nc = int(input('Choose a category number ')) #19
d_urls = []
c_names = []
links2 = links[nc-1:nc]
names2 = name[nc-1:nc]
for i, link in enumerate(links2):
  c_names.append(names2[i])
  d_urls.append(main_url + link)
print(c_names)
len(c_names), len(d_urls)

Choose a category number 19
['Malattie di endocrinologia e diabetologia']


(1, 1)

## SCRAPING DISEASE PAGE

In [6]:
d_names_list = []
ante_urls_list = []
for c, d_url in enumerate(d_urls):
  dScraper = Scraper(d_url)
  dScraper.req_tags('item-page', 'a')
  name, links = dScraper.req_item('href')#, dScraper.tags[1:2])
  #print(len(links))
  ante_urls = []
  d_names = []
  for d, link in enumerate(links):
    d_names.append(name[d])
    ante_urls.append(main_url + link)
  d_names_list.append(d_names)
  ante_urls_list.append(ante_urls)
  print('d =', len(d_names), d_names)
print('c =', len(d_names_list))

d = 28 ['Acromegalia', 'Anoressia', 'Bulimia', 'Deficit di ormone della crescita', 'Diabete mellito (tipo 1; tipo 2)', 'Disturbo di Identità di Genere (DIG)', 'Fenilchetonuria', 'Gozzo nodulare tiroideo', 'Iperaldosteronismo', 'Iperinsulinemia', 'Iperparatiroidismo', 'Ipertiroidismo', 'Ipocalcemia', 'Ipoglicemia', 'Ipogonadismo', 'Ipopituitarismo', 'Ipotiroidismo', 'Malattia di Addison', 'Peritonite', 'Pubertà precoce', 'Sindrome Adreno Genitale', 'Sindrome di Pallister-Hall', 'Sindrome di Poland', 'Sindrome di Turner', 'Sindrome metabolica', 'Tiroidite (acuta; cronica)', 'Tiroidite di De Quervain (o tiroidite granulomatosa a cellule giganti)', 'Tiroidite di Hashimoto']
c = 1


In [7]:
print(ante_urls_list)

print(d_names_list[0][0])

[['https://www.qsalute.it/recensioni/acromegalia/', 'https://www.qsalute.it/recensioni/anoressia/', 'https://www.qsalute.it/recensioni/bulimia/', 'https://www.qsalute.it/recensioni/deficit-di-ormone-della-crescita/', 'https://www.qsalute.it/recensioni/diabete-mellito-tipo-1-tipo-2/', 'https://www.qsalute.it/recensioni/disturbo-di-identita-di-genere-dig/', 'https://www.qsalute.it/recensioni/fenilchetonuria/', 'https://www.qsalute.it/recensioni/gozzo-multinodulare-tiroideo/', 'https://www.qsalute.it/recensioni/iperaldosteronismo/', 'https://www.qsalute.it/recensioni/iperinsulinemia/', 'https://www.qsalute.it/recensioni/iperparatiroidismo/', 'https://www.qsalute.it/recensioni/ipertiroidismo/', 'https://www.qsalute.it/recensioni/ipocalcemia/', 'https://www.qsalute.it/recensioni/ipoglicemia/', 'https://www.qsalute.it/recensioni/ipogonadismo/', 'https://www.qsalute.it/recensioni/ipopituitarismo/', 'https://www.qsalute.it/recensioni/ipotiroidismo/', 'https://www.qsalute.it/recensioni/malattia

In [8]:
a_names_list = []
all_rev_url_list = []
for c, ante_urls in enumerate(ante_urls_list):
  rev_url_list = []
  a_names = []
  for d, ante_url in enumerate(ante_urls):
    page_url_list = []
    a_names.append(d_names_list[c][d])
    for i in range(1,1000):
      nav_url = ante_url + '?page=' + str(i)
      try:
        anteScraper = Scraper(nav_url)
        anteScraper.req_tags('jrReviewList', tag='a', attributes={"class": "jrButton"})
        _, rev_url = anteScraper.req_item('href', ifname="Leggi Recensione")
      except Exception as e:
        #print(e)
        #print(d_names_list[c][d], i-1)
        break
      page_url_list.append(rev_url)
    rev_url_list.append(page_url_list)
    print(str(d) + '. ' + ante_url + ': FINISHED')
  all_rev_url_list.append(rev_url_list)
  a_names_list.append(a_names)
  print(c_names[c] + ': FINISHED')
#'c: ', len(all_rev_url_list), 'd: ', len(all_rev_url_list[0]), 'a: ', len(all_rev_url_list[0][1]), 'r: ', all_rev_url_list[0][1][0][0]

0. https://www.qsalute.it/recensioni/acromegalia/: FINISHED
1. https://www.qsalute.it/recensioni/anoressia/: FINISHED
2. https://www.qsalute.it/recensioni/bulimia/: FINISHED
3. https://www.qsalute.it/recensioni/deficit-di-ormone-della-crescita/: FINISHED
4. https://www.qsalute.it/recensioni/diabete-mellito-tipo-1-tipo-2/: FINISHED
5. https://www.qsalute.it/recensioni/disturbo-di-identita-di-genere-dig/: FINISHED
6. https://www.qsalute.it/recensioni/fenilchetonuria/: FINISHED
7. https://www.qsalute.it/recensioni/gozzo-multinodulare-tiroideo/: FINISHED
8. https://www.qsalute.it/recensioni/iperaldosteronismo/: FINISHED
9. https://www.qsalute.it/recensioni/iperinsulinemia/: FINISHED
10. https://www.qsalute.it/recensioni/iperparatiroidismo/: FINISHED
11. https://www.qsalute.it/recensioni/ipertiroidismo/: FINISHED
12. https://www.qsalute.it/recensioni/ipocalcemia/: FINISHED
13. https://www.qsalute.it/recensioni/ipoglicemia/: FINISHED
14. https://www.qsalute.it/recensioni/ipogonadismo/: FINIS

## SCRAPING REVIEWS

In [9]:
c_superDF = []
for c, d_list in enumerate(all_rev_url_list):
  #print(c, d_list)
  d_superDF = []
  for d, a_list in enumerate(d_list):
    #print(c, d, a_list)
    p_superDF = []
    for p, p_list in enumerate(a_list):
      #print(c, d, p, p_list)
      superDF = []
      for r, rev_url in enumerate(p_list):
        #print(c, d, p, r, rev_url, d_names_list[c][d])
        rev = Review(rev_url, d_names_list[c][d])
        df = rev.set_review()
        superDF.append(df)
      #print('s', superDF, '\n')
      p_superDF.append(pd.concat(superDF, ignore_index=True))
    #print('p', p_superDF, '\n')
    try:
      d_superDF.append(pd.concat(p_superDF, ignore_index=True))
    except Exception as e:
        print(e)
        print(c, c_names[c], d, d_names_list[c][d])
    print(d, d_names_list[c][d])
  c_superDF.append((pd.concat(d_superDF, ignore_index=True)))
  #print(c_superDF)
  c_superDF[c].to_csv(path + c_names[c] +'.csv', sep='\t', index=False)
  print(c_names[c] + ': FINISHED')

0 Acromegalia
1 Anoressia
2 Bulimia
3 Deficit di ormone della crescita
4 Diabete mellito (tipo 1; tipo 2)
5 Disturbo di Identità di Genere (DIG)
6 Fenilchetonuria
7 Gozzo nodulare tiroideo
8 Iperaldosteronismo
9 Iperinsulinemia
10 Iperparatiroidismo
11 Ipertiroidismo
12 Ipocalcemia
13 Ipoglicemia
14 Ipogonadismo
15 Ipopituitarismo
16 Ipotiroidismo
17 Malattia di Addison
18 Peritonite
19 Pubertà precoce
20 Sindrome Adreno Genitale
21 Sindrome di Pallister-Hall
22 Sindrome di Poland
23 Sindrome di Turner
24 Sindrome metabolica
25 Tiroidite (acuta; cronica)
26 Tiroidite di De Quervain (o tiroidite granulomatosa a cellule giganti)
27 Tiroidite di Hashimoto
Malattie di endocrinologia e diabetologia: FINISHED


## List of diseases with no reviews (May, 26th 2020)
1. Cervello:
  * 27 Craniosinostosi 
  * 55 Fistola liquorale
  * 198 Malattia di Rendu-Osler-Weber (o telangectasia emorragica ereditaria HHT)
2. Pelle.
3. Cuore:
  * 14 Atresia polmonare a setto intatto
  * 32 Difetto interatriale (DIA): informazioni
4. Urologia:
  * 72 Uretere ectopico
5. Ginecologia.
6. Senologia.
7. Oncologia:
  * 61 Tumore del colon-retto (informazioni)
8. Ematologia.
9. Reumatologia.
10. Pneumologia.
11. Gastro.
12. Chirurgia Vascolare.
13. Chirurgia Toracica:
  * 6 Carcinoide polmonare (tipico; atipico)
14. Chirurgia Maxillo-facciale.
15. Chirurgia plastica.
16. Oculistica.
17. Infettive.
18. Otorino.
19. Endocrino.
